In [2]:
import duckdb
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
import sqlite3




real_files = "read_parquet('results/processed/real/**/*.parquet')"
synth_files = "read_parquet('results/processed/synth/**/*.parquet')"


In [18]:
query_real = f"""
    SELECT 
        gene_id,
        COUNT(*) FILTER (WHERE is_gene_upregulated = TRUE) as upregulated,
        COUNT(*) FILTER (WHERE is_gene_upregulated = FALSE) as downregulated
    FROM {real_files}
    GROUP BY gene_id
"""

result_real = duckdb.sql(query_real).df()


In [17]:

query_synth = f"""
    SELECT 
        gene_id,
        COUNT(*) FILTER (WHERE is_gene_upregulated = TRUE) as upregulated,
        COUNT(*) FILTER (WHERE is_gene_upregulated = FALSE) as downregulated
    FROM {synth_files}
    GROUP BY gene_id
"""


result_synth = duckdb.sql(query_synth).df()


# crossection

In [19]:
common_genes = list(set(result_real.gene_id)  & set(result_synth.gene_id))
result_real = result_real[result_real.gene_id.isin(common_genes)]
result_synth = result_synth[result_synth.gene_id.isin(common_genes)]


# division by 10

In [20]:
result_synth["upregulated"] = result_synth["upregulated"] / 10
result_synth["downregulated"] = result_synth["downregulated"] / 10

In [ ]:
values = pd.merge(result_real, result_synth, on="gene_id", suffixes=["_real", "_synth"])

In [30]:
def calculate_log2_odds_ratio(a, b, c, d, k=0.5):
    # a, b, c, d are the four cells of the 2x2 contingency table
    # k is the smoothing constant
    #
    odds_ratio = ((a + k) * (d + k)) / ((b + k) * (c + k))
    return np.log2(odds_ratio)

# laplace smoothing with k=0.5 (Jeffreys prior)
values['log2_odds_ratio'] = values.apply(lambda row: calculate_log2_odds_ratio(
    row['upregulated_real'], 
    row['downregulated_real'], 
    row['upregulated_synth'], 
    row['downregulated_synth']
), axis=1)

values

,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio
0,ENSG00000148482,2419,1267,1973.3,835.4,-0.306850
1,ENSG00000077327,2485,1041,2016.7,1017.1,0.267690
2,ENSG00000111696,988,382,1146.2,435.8,-0.024314
3,ENSG00000080910,4384,1966,3176.6,1542.5,0.114810
4,ENSG00000134376,6788,3224,6082.3,2744.7,-0.073807
...,...,...,...,...,...,...
31424,ENSG00000177535,26,13,14.7,7.3,0.010508
31425,ENSG00000130656,3,9,1.2,3.6,-0.170483
31426,ENSG00000184761,10,4,177.4,26.2,-1.513762
31427,ENSG00000229348,1,17,8.4,9.4,-3.390697


In [ ]:
def perform_fisher_test_vectorized(df, pseudocount=0.5):
    # Add pseudocount to the table
    table = np.array([
        [df['upregulated_real'] + pseudocount, df['downregulated_real'] + pseudocount],
        [df['upregulated_synth'] + pseudocount, df['downregulated_synth'] + pseudocount]
    ]).transpose((2, 0, 1))  # reshape for 2x2 tables

    p_values = np.zeros(len(df))

    for i in range(len(df)):
        _, p_values[i] = fisher_exact(table[i])

    df['p_value'] = p_values
    df['p_adj'] = multipletests(p_values, method='fdr_bh')[1]
    
    return df
def add_z_score(df):
    # Calculate mean and standard deviation of log2 odds ratios
    mean_log2or = df['log2_odds_ratio'].mean()
    std_log2or = df['log2_odds_ratio'].std()
    
    # Calculate Z-score
    df['z_score'] = (df['log2_odds_ratio'] - mean_log2or) / std_log2or
    
    return df


df = perform_fisher_test_vectorized(values)
df = add_z_score(df)
df.head()

In [ ]:
from scripts.pyensembl_operations import import_pyensembl
g37 = import_pyensembl(37)

gene_names = {gene.gene_id: gene.gene_name for gene in g37.genes()}
print(f"Total genes: {len(gene_names)}")
print("\nFirst 3 examples:")
for gene_id, name in list(gene_names.items())[:3]:
    print(f"{gene_id}: {name}")

df["gene_name"] = df["gene_id"].map(gene_names)

df.head()

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


Total genes: 63677

First 3 examples:
ENSG00000000003: TSPAN6
ENSG00000000005: TNMD
ENSG00000000419: DPM1


,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio,p_value,p_adj,z_score,gene_name
0,ENSG00000148482,2419,1267,1973.3,835.4,-0.306850,0.000074,0.000477,-0.357743,SLC39A12
1,ENSG00000077327,2485,1041,2016.7,1017.1,0.267690,0.000522,0.002567,0.085154,SPAG6
2,ENSG00000111696,988,382,1146.2,435.8,-0.024314,0.869021,0.997825,-0.139944,NT5DC3
3,ENSG00000080910,4384,1966,3176.6,1542.5,0.114810,0.054780,0.126744,-0.032697,CFHR2
4,ENSG00000134376,6788,3224,6082.3,2744.7,-0.073807,0.106063,0.214494,-0.178096,CRB1


In [49]:
from scripts.pyensembl_operations import import_pyensembl
g37 = import_pyensembl(37)

biotypes = {gene.gene_id: gene.biotype for gene in g37.genes()}
print(f"Total genes: {len(biotypes)}")
print("\nFirst 3 examples:")
for gene_id, name in list(biotypes.items())[:3]:
    print(f"{gene_id}: {name}")

df["biotype"] = df["gene_id"].map(biotypes)

df.head()

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


Total genes: 63677

First 3 examples:
ENSG00000000003: protein_coding
ENSG00000000005: protein_coding
ENSG00000000419: protein_coding


,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio,p_value,p_adj,z_score,gene_name,biotype
0,ENSG00000148482,2419,1267,1973.3,835.4,-0.306850,0.000074,0.000477,-0.357743,SLC39A12,protein_coding
1,ENSG00000077327,2485,1041,2016.7,1017.1,0.267690,0.000522,0.002567,0.085154,SPAG6,protein_coding
2,ENSG00000111696,988,382,1146.2,435.8,-0.024314,0.869021,0.997825,-0.139944,NT5DC3,protein_coding
3,ENSG00000080910,4384,1966,3176.6,1542.5,0.114810,0.054780,0.126744,-0.032697,CFHR2,protein_coding
4,ENSG00000134376,6788,3224,6082.3,2744.7,-0.073807,0.106063,0.214494,-0.178096,CRB1,protein_coding


In [33]:
# df.to_csv("results/the_final_data.csv", index=False)
df = pd.read_csv("results/the_final_data.csv")

# drop entries where gene_id is not_found
df = df[df["gene_id"] != "not_found"]

# add filter col
filter_logic = (abs(df['log2_odds_ratio']) > 0.32) & (df['p_adj'] < 0.05)
df["is_significant"] = filter_logic

# add genes table
sqlite_conn = sqlite3.connect('data/mirscribe.db')
genes = pd.read_sql('SELECT * FROM genes', sqlite_conn)
sqlite_conn.close()

# add genes table details
cols_to_merge = ['gene_id', 'is_oncogene_oncokb', 'is_tsupp_oncokb',
       'is_driver_intogen', 'tier_cosmic', 'is_hallmark_cosmic',
       'is_tsupp_cosmic', 'is_oncogene_cosmic', 'is_oncogene_consensus',
       'is_tsupp_consensus', 'is_gene_of_interest', 'cancer_gene_role']

enriched_df = pd.merge(df, genes[cols_to_merge], how="left", on="gene_id" )

In [57]:
enriched_df

,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio,p_value,p_adj,z_score,gene_name,...,is_tsupp_oncokb,is_driver_intogen,tier_cosmic,is_hallmark_cosmic,is_tsupp_cosmic,is_oncogene_cosmic,is_oncogene_consensus,is_tsupp_consensus,is_gene_of_interest,cancer_gene_role
0,ENSG00000148482,2419,1267,1973.3,835.4,-0.306850,0.000074,0.000477,-0.357743,SLC39A12,...,0,0,0,0,0,0,0,0,0,neither
1,ENSG00000077327,2485,1041,2016.7,1017.1,0.267690,0.000522,0.002567,0.085154,SPAG6,...,0,0,0,0,0,0,0,0,0,neither
2,ENSG00000111696,988,382,1146.2,435.8,-0.024314,0.869021,0.997825,-0.139944,NT5DC3,...,0,0,0,0,0,0,0,0,0,neither
3,ENSG00000080910,4384,1966,3176.6,1542.5,0.114810,0.054780,0.126744,-0.032697,CFHR2,...,0,0,0,0,0,0,0,0,0,neither
4,ENSG00000134376,6788,3224,6082.3,2744.7,-0.073807,0.106063,0.214494,-0.178096,CRB1,...,0,0,0,0,0,0,0,0,0,neither
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31423,ENSG00000177535,26,13,14.7,7.3,0.010508,1.000000,1.000000,-0.113100,OR2B11,...,0,0,0,0,0,0,0,0,0,neither
31424,ENSG00000130656,3,9,1.2,3.6,-0.170483,1.000000,1.000000,-0.252621,HBZ,...,0,0,0,0,0,0,0,0,0,neither
31425,ENSG00000184761,10,4,177.4,26.2,-1.513762,0.109875,0.220599,-1.288118,AC013269.5,...,0,0,0,0,0,0,0,0,0,neither
31426,ENSG00000229348,1,17,8.4,9.4,-3.390697,0.007230,0.023846,-2.734996,HYI-AS1,...,0,0,0,0,0,0,0,0,0,neither


In [ ]:
query_real = f"""
    SELECT 
        *
    FROM {real_files}
    LIMIT   5
"""

result_query = duckdb.sql(query_real).df()
result_query.head()

,id,wt_prediction,mut_prediction,pred_difference,vcf_id,mirna_accession,gene_id,is_intron,mutation_context,mutsig_key,is_gene_upregulated,experiment,mutsig,gene_name,biotype,cancer_type
0,PD10010a_10_100900726_G_A_MIMAT0000257,0.684779,0.269260,-0.416,PD10010a,MIMAT0000257,ENSG00000172987,True,C[C>T]G,PD10010a_C[C>T]G,True,0,SBS1,HPSE2,protein_coding,nnn
1,PD10010a_10_100900726_G_A_MIMAT0000441,0.628840,0.424571,-0.204,PD10010a,MIMAT0000441,ENSG00000172987,True,C[C>T]G,PD10010a_C[C>T]G,True,0,SBS1,HPSE2,protein_coding,nnn
2,PD10010a_10_100900726_G_A_MIMAT0001635,0.682595,0.397291,-0.285,PD10010a,MIMAT0001635,ENSG00000172987,True,C[C>T]G,PD10010a_C[C>T]G,True,0,SBS1,HPSE2,protein_coding,nnn
3,PD10010a_10_100900726_G_A_MIMAT0003264,0.831073,0.324415,-0.507,PD10010a,MIMAT0003264,ENSG00000172987,True,C[C>T]G,PD10010a_C[C>T]G,True,0,SBS1,HPSE2,protein_coding,nnn
4,PD10010a_10_100900726_G_A_MIMAT0003284,0.330688,0.552029,0.221,PD10010a,MIMAT0003284,ENSG00000172987,True,C[C>T]G,PD10010a_C[C>T]G,False,0,SBS1,HPSE2,protein_coding,nnn


# sqlite

In [8]:

conn = sqlite3.connect('data/mirscribe.db')
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

transcripts
gsea
mirnas
genes
results
real_results
synth_results


In [65]:
pd.read_sql_query("SELECT * FROM predictions", conn)

,id,wt_prediction,mut_prediction,pred_difference,vcf_id,mirna_accession,gene_id,mutation_context,mutsig,is_intron,is_gain,is_gene_upregulated
0,PD4005a_1_12844091_G_T_MIMAT0000265,0.411319,0.724802,0.313,PD4005a,MIMAT0000265,None,A[C>A]A,SBS3,0,1,0
1,PD4005a_1_12844091_G_T_MIMAT0003301,0.289311,0.662916,0.374,PD4005a,MIMAT0003301,None,A[C>A]A,SBS3,0,1,0
2,PD4005a_1_12844091_G_T_MIMAT0004517,0.351466,0.623310,0.272,PD4005a,MIMAT0004517,None,A[C>A]A,SBS3,0,1,0
3,PD4005a_1_12844091_G_T_MIMAT0005882,0.550857,0.279106,-0.272,PD4005a,MIMAT0005882,None,A[C>A]A,SBS3,0,0,1
4,PD4005a_1_12844091_G_T_MIMAT0005933,0.650570,0.375474,-0.275,PD4005a,MIMAT0005933,None,A[C>A]A,SBS3,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3608886,PD3890a_3_9912562_G_T_MIMAT0027509,0.437611,0.875881,0.438,PD3890a,MIMAT0027509,ENSG00000187288,T[C>A]C,SBS3,1,1,0
3608887,PD3890a_3_9912562_G_T_MIMAT0027535,0.305159,0.604791,0.300,PD3890a,MIMAT0027535,ENSG00000187288,T[C>A]C,SBS3,1,1,0
3608888,PD3890a_3_9912562_G_T_MIMAT0027581,0.402005,0.725282,0.323,PD3890a,MIMAT0027581,ENSG00000187288,T[C>A]C,SBS3,1,1,0
3608889,PD3890a_3_9912562_G_T_MIMAT0027627,0.247087,0.528400,0.281,PD3890a,MIMAT0027627,ENSG00000187288,T[C>A]C,SBS3,1,1,0


In [1]:
import duckdb
import sqlite3
import glob
from tqdm import tqdm

# Connect to databases
sqlite_conn = sqlite3.connect('data/mirscribe.db')
duck_conn = duckdb.connect()

# Get list of all parquet files
parquet_files = glob.glob('results/processed/synth/**/*.parquet', recursive=True)

# Process first file to create table and set schema
print("Creating table with first file...")
first_file = parquet_files[0]
query = f"""
    SELECT *
    FROM read_parquet('{first_file}')
    LIMIT 1000
"""
# Create table schema
df_schema = duck_conn.sql(query).df()
df_schema.to_sql('synth_results', sqlite_conn, if_exists='replace', index=False)
sqlite_conn.commit()

# Process each file in chunks
chunk_size = 50000  # Adjust based on your memory constraints

print("Processing files...")
for file in tqdm(parquet_files):
    # Count total rows in parquet file
    total_rows = duck_conn.sql(f"SELECT COUNT(*) FROM read_parquet('{file}')")
    total_rows = total_rows.fetchone()[0]
    
    # Process file in chunks
    for offset in range(0, total_rows, chunk_size):
        query = f"""
            SELECT *
            FROM read_parquet('{file}')
            LIMIT {chunk_size}
            OFFSET {offset}
        """
        chunk_df = duck_conn.sql(query).df()
        
        # Write chunk to SQLite
        chunk_df.to_sql('synth_results', 
                       sqlite_conn, 
                       if_exists='append', 
                       index=False)
        
    sqlite_conn.commit()  # Commit after each file

# Verify the data
cursor = sqlite_conn.cursor()
cursor.execute("SELECT COUNT(*) FROM synth_results")
print(f"\nTotal rows in database: {cursor.fetchone()[0]}")

# Close connections
sqlite_conn.close()
duck_conn.close()


Creating table with first file...
Processing files...


 74%|███████▍  | 4166/5610 [1:21:04<28:06,  1.17s/it]  


OperationalError: database or disk is full

In [1]:
import sqlite3

def backup_specific_tables(source_path, destination_path, tables):
    try:
        # Connect to source and destination
        source = sqlite3.connect(source_path)
        destination = sqlite3.connect(destination_path)
        
        for table in tables:
            try:
                print(f"\nProcessing table: {table}")
                
                # Drop table if exists in destination
                destination.execute(f"DROP TABLE IF EXISTS {table}")
                destination.commit()
                
                # Get the table creation SQL
                query = f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table}'"
                create_stmt = source.execute(query).fetchone()
                
                if create_stmt:
                    # Create table in new database
                    destination.execute(create_stmt[0])
                    
                    # Get column names
                    cursor = source.execute(f"SELECT * FROM {table} LIMIT 1")
                    columns = [description[0] for description in cursor.description]
                    placeholders = ','.join(['?' for _ in columns])
                    
                    # Copy data
                    data = source.execute(f"SELECT * FROM {table}").fetchall()
                    if data:
                        insert_sql = f"INSERT INTO {table} VALUES ({placeholders})"
                        destination.executemany(insert_sql, data)
                    
                    destination.commit()
                    print(f"Successfully backed up table: {table} with {len(data)} rows")
                    
            except sqlite3.Error as e:
                print(f"Error backing up table {table}: {e}")
                continue
                
        print("\nBackup process completed")
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        source.close()
        destination.close()

# Tables to backup
tables_to_backup = [
    'transcripts',
    'gsea',
    'mirnas',
    'genes',
    'results'
]

# Execute backup
backup_specific_tables(
    'data/mirscribe.db', 
    'data/mirscribe_backup.db',
    tables_to_backup
)



Processing table: transcripts
Successfully backed up table: transcripts with 252989 rows

Processing table: gsea
Successfully backed up table: gsea with 4384 rows

Processing table: mirnas
Successfully backed up table: mirnas with 2656 rows

Processing table: genes
Successfully backed up table: genes with 57736 rows

Processing table: results
Successfully backed up table: results with 31428 rows

Backup process completed
